<a href="https://colab.research.google.com/github/anetas60/kurs2/blob/main/supervised/01_basic/03_future_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import sklearn
sklearn.__version__

'1.6.1'

Załadowanie danych

In [2]:

def fetch_financial_data(company='AMZN'):
    """
    This function fetches stock market quotations.
    """
    import pandas_datareader.data as web
    return web.DataReader(name=company, data_source='stooq')

df_raw = fetch_financial_data()
df_raw.head()


,Open,High,Low,Close,Volume
Date,,,,,
2025-10-17,214.560,214.80,211.0300,213.04,45986944
2025-10-16,215.670,218.59,212.8101,214.47,42414591
2025-10-15,216.620,217.71,212.6600,215.57,45909469
2025-10-14,215.555,219.32,212.6000,216.39,45665580
2025-10-13,217.700,220.68,217.0400,220.07,37809650


Utworzenie kopii danych

In [3]:
df = df_raw.copy()
df = df[:5]
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5 entries, 2025-10-17 to 2025-10-13
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    5 non-null      float64
 1   High    5 non-null      float64
 2   Low     5 non-null      float64
 3   Close   5 non-null      float64
 4   Volume  5 non-null      int64  
dtypes: float64(4), int64(1)
memory usage: 240.0 bytes


Generowanie nowych zmiennych

In [4]:
df['day'] = df.index.day
df['month'] = df.index.month
df['year'] = df.index.year
df

,Open,High,Low,Close,Volume,day,month,year
Date,,,,,,,,
2025-10-17,214.560,214.80,211.0300,213.04,45986944,17,10,2025
2025-10-16,215.670,218.59,212.8101,214.47,42414591,16,10,2025
2025-10-15,216.620,217.71,212.6600,215.57,45909469,15,10,2025
2025-10-14,215.555,219.32,212.6000,216.39,45665580,14,10,2025
2025-10-13,217.700,220.68,217.0400,220.07,37809650,13,10,2025


Dyskretyzacja zmiennej ciągłej - zamiana zmiennej ciągłej na kategoryczną

In [5]:
df = pd.DataFrame(data={'height': [175., 178.5, 185., 191., 184.5, 168.]})
df

,height
0,175.0
1,178.5
2,185.0
3,191.0
4,184.5
5,168.0


In [6]:
df['height_cat'] = pd.cut(x=df.height, bins=3)
df

,height,height_cat
0,175.0,"(167.977, 175.667]"
1,178.5,"(175.667, 183.333]"
2,185.0,"(183.333, 191.0]"
3,191.0,"(183.333, 191.0]"
4,184.5,"(183.333, 191.0]"
5,168.0,"(167.977, 175.667]"


In [7]:
df['height_cat'] = pd.cut(x=df.height, bins=(160, 175, 180, 195))
df

,height,height_cat
0,175.0,"(160, 175]"
1,178.5,"(175, 180]"
2,185.0,"(180, 195]"
3,191.0,"(180, 195]"
4,184.5,"(180, 195]"
5,168.0,"(160, 175]"


In [8]:
df['height_cat'] = pd.cut(x=df.height, bins=(160, 175, 180, 195), labels=['small', 'medium', 'hight'])
df

,height,height_cat
0,175.0,small
1,178.5,medium
2,185.0,hight
3,191.0,hight
4,184.5,hight
5,168.0,small


In [9]:
pd.get_dummies(df, drop_first=True, prefix='height', dtype=int)

,height,height_medium,height_hight
0,175.0,0,0
1,178.5,1,0
2,185.0,0,1
3,191.0,0,1
4,184.5,0,1
5,168.0,0,0


Ekstrakcja cech

In [10]:
df = pd.DataFrame(data={'lang': [['PL', 'ENG'], ['GER', 'ENG', 'PL', 'FRA'], ['RUS']]})
df

,lang
0,"[PL, ENG]"
1,"[GER, ENG, PL, FRA]"
2,[RUS]


In [11]:
df['lang_number'] = df['lang'].apply(len)
df

,lang,lang_number
0,"[PL, ENG]",2
1,"[GER, ENG, PL, FRA]",4
2,[RUS],1


In [12]:
df['PL_flag'] = df['lang'].apply(lambda x: 1 if 'PL' in x else 0)
df

,lang,lang_number,PL_flag
0,"[PL, ENG]",2,1
1,"[GER, ENG, PL, FRA]",4,1
2,[RUS],1,0


In [13]:
df = pd.DataFrame(data={'website': ['wp.pl', 'onet.pl', 'google.com']})
df

,website
0,wp.pl
1,onet.pl
2,google.com


In [14]:
df.website.str.split('.', expand=True)

,0,1
0,wp,pl
1,onet,pl
2,google,com


In [15]:
from typing_extensions import dataclass_transform
new = df.website.str.split('.', expand=True)
df['portal'] = new[0]
df['extension'] = new[1]
dataclass_transform
df

,website,portal,extension
0,wp.pl,wp,pl
1,onet.pl,onet,pl
2,google.com,google,com
